[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/qgis-basemaps/blob/main/qgis_basemaps.ipynb)

In [ ]:
# %pip install leafmap

In [ ]:
import leafmap
import pandas as pd

In [ ]:
maps = leafmap.basemaps

In [ ]:
sources = []
for basemap in maps:
    if maps[basemap]['type'] == 'xyz' and ("Stadia" not in basemap):
        source = []
        source.append("connections-xyz")
        source.append(basemap)
        source.append("")
        source.append("")
        source.append(maps[basemap]['attribution'][:50])
        source.append(maps[basemap]['url'].replace("&key=YOUR-API-KEY", ""))
        source.append("")
        source.append("30")
        source.append("0")
        sources.append(source)
xyz_count = len(sources)

In [ ]:
print(f"The number of XYZ basemaps is {len(sources)}.")

In [ ]:
# for basemap in maps:
#     if maps[basemap]['type'] == 'wms':
#         source = []
#         source.append("connections-wms")
#         source.append(basemap)
#         source.append(maps[basemap]['url'].replace("?", ""))
#         source.append(maps[basemap]['attribution'][:50])
#         source.append("30")
#         source.append("0")
#         sources.append(source)
# wms_count = len(sources) - xyz_count    

In [ ]:
df = pd.read_csv("wms_basemaps.csv")
df

In [ ]:
for i, row in df.iterrows():
    source = []
    source.append("connections-wms")
    source.append(row['name'].strip())
    source.append(row['url'].strip())
    source.append(row['referer'].strip())
    source.append("30".strip())
    source.append("0".strip())
    sources.append(source)
wms_count = len(sources) - xyz_count

In [ ]:
print(f"The number of WMS basemaps is {wms_count}.")

In [ ]:
with open("qgis_basemaps.py", "w") as f:

    header = """
# This script is inspired by Klas Karlsson at https://bit.ly/4eDbx1O. Credit to Klas Karlsson.
# I expaned the script to include more XYZ basemaps from xyzservices and a lot of WMS basemaps.\n\n
"""
    f.write(header)

    f.write("from qgis.PyQt.QtCore import QSettings\n")
    f.write("from qgis.utils import iface\n")
    f.write("print('Adding basemaps to QGIS now. Please wait ...')\n\n")
    f.write("sources = []\n")

    bing_maps = [
        "connections-xyz",
        "Bing VirtualEarth",
        "",
        "",
        "Microsoft",
        "http://ecn.t3.tiles.virtualearth.net/tiles/a{q}.jpeg?g=1",
        "",
        "30",
        "1",
    ]
    line = f"sources.append({str(bing_maps)})\n"
    f.write(line)

    for source in sources:
        line = f"sources.append({str(source)})\n"
        f.write(line)

    footer = """
for source in sources:
    connectionType = source[0]
    connectionName = source[1]
    
    if connectionType == "connections-xyz":
        QSettings().setValue(f"qgis/{connectionType}/{connectionName}/authcfg", source[2])
        QSettings().setValue(f"qgis/{connectionType}/{connectionName}/password", source[3])
        QSettings().setValue(f"qgis/{connectionType}/{connectionName}/referer", source[4])
        QSettings().setValue(f"qgis/{connectionType}/{connectionName}/url", source[5])
        QSettings().setValue(f"qgis/{connectionType}/{connectionName}/username", source[6])
        QSettings().setValue(f"qgis/{connectionType}/{connectionName}/zmax", source[7])
        QSettings().setValue(f"qgis/{connectionType}/{connectionName}/zmin", source[8])
    elif connectionType == "connections-wms":
        QSettings().setValue(f"qgis/{connectionType}/{connectionName}/url", source[2])
        QSettings().setValue(f"qgis/{connectionType}/{connectionName}/referer", source[3])
        QSettings().setValue(f"qgis/{connectionType}/{connectionName}/zmax", source[4])
        QSettings().setValue(f"qgis/{connectionType}/{connectionName}/zmin", source[5])
    

iface.reloadConnections()

print("All basemaps have been added. Please restart QGIS.")
"""
    f.write(footer)